<a href="https://colab.research.google.com/github/thenukan/Covid_19-detection---using-Xray/blob/main/PDF_insight.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [21]:
import tabula
import pandas as pd
import PyPDF2
import json
dictonary_main={}
pdf_path="/content/sample.pdf"
file = open(pdf_path, 'rb')
readpdf = PyPDF2.PdfFileReader(file)
totalpages = readpdf.numPages
for page in range(1,totalpages+1):
    dfs=tabula.read_pdf(pdf_path,pages=str(page))
    for tables in range(len(dfs)):
        dictonary={}
        head=dfs[tables].columns.tolist()
        dictonary["header"]=head
        products_list = dfs[tables].values.tolist()
        products_list_str=[]
        for obj in products_list:
            obj=[str(i) for i in obj]
            products_list_str.append(obj)
        
        
        dictonary["data"]=products_list_str
        dictonary_main[str(page)+str(tables)]=dictonary

with open('/content/app.json', 'w') as fp:
    json.dump(dictonary_main, fp)


/usr/local/lib/python3.7/dist-packages/PyPDF2/_reader.py:1362: PdfReadWarning: Xref table not zero-indexed. ID numbers for objects will be corrected.
  PdfReadWarning,


In [6]:
# Make sure you have a GPU running
!nvidia-smi

NVIDIA-SMI has failed because it couldn't communicate with the NVIDIA driver. Make sure that the latest NVIDIA driver is installed and running.



In [7]:
# Install the latest release of Haystack in your own environment
#! pip install farm-haystack

# Install the latest master of Haystack
!pip install --upgrade pip
!pip install git+https://github.com/deepset-ai/haystack.git#egg=farm-haystack[colab]

# The TaPAs-based TableReader requires the torch-scatter library
import torch

version = torch.__version__
!pip install torch-scatter -f https://data.pyg.org/whl/torch-{version}.html

# Install pygraphviz for visualization of Pipelines
!apt install libgraphviz-dev
!pip install pygraphviz

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 2.1 MB 5.5 MB/s 
  Attempting uninstall: pip
    Found existing installation: pip 21.1.3
    Uninstalling pip-21.1.3:
      Successfully uninstalled pip-21.1.3
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/deepset-ai/haystack.git to /tmp/pip-install-c8axg2h6/farm-haystack_6ed54e8fa1774801b2d1db81affb1844
  Running command git clone --filter=blob:none --quiet https://github.com/deepset-ai/haystack.git /tmp/pip-install-c8axg2h6/farm-haystack_6ed54e8fa1774801b2d1db81affb1844
  Resolved https://github.com/deepset-ai/haystack.git to commit a26c0429941b2c528406bac130483937f0dba50d
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.8

In [8]:
# Recommended: Start Elasticsearch using Docker via the Haystack utility function
from haystack.utils import launch_es

launch_es()

INFO - haystack.modeling.model.optimization -  apex not found, won't use it. See https://nvidia.github.io/apex/
ERROR - root -  Failed to import 'magic' (from 'python-magic' and 'python-magic-bin' on Windows). FileTypeClassifier will not perform mimetype detection on extensionless files. Please make sure the necessary OS libraries are installed if you need this functionality.
WARNING - haystack.utils.doc_store -  Tried to start Elasticsearch through Docker but this failed. It is likely that there is already an existing Elasticsearch instance running. 


In [9]:
# In Colab / No Docker environments: Start Elasticsearch from source
! wget https://artifacts.elastic.co/downloads/elasticsearch/elasticsearch-7.9.2-linux-x86_64.tar.gz -q
! tar -xzf elasticsearch-7.9.2-linux-x86_64.tar.gz
! chown -R daemon:daemon elasticsearch-7.9.2

import os
from subprocess import Popen, PIPE, STDOUT

es_server = Popen(
    ["elasticsearch-7.9.2/bin/elasticsearch"], stdout=PIPE, stderr=STDOUT, preexec_fn=lambda: os.setuid(1)  # as daemon
)
# wait until ES has started
! sleep 30

In [12]:
# Connect to Elasticsearch
from haystack.document_stores import ElasticsearchDocumentStore

document_index = "document"
document_store = ElasticsearchDocumentStore(host="localhost", username="", password="", index=document_index)

In [23]:
import json
from haystack import Document
import pandas as pd


def read_tables(filename):
    processed_tables = []
    with open(filename) as tables:
        tables = json.load(tables)
        for key, table in tables.items():
            current_columns = table["header"]
            current_rows = table["data"]
            current_df = pd.DataFrame(columns=current_columns, data=current_rows)
            document = Document(content=current_df, content_type="table", id=key)
            processed_tables.append(document)

    return processed_tables

doc_dir = "/content"
tables = read_tables(f"{doc_dir}/app.json")
document_store.write_documents(tables, index=document_index)

# Showing content field and meta field of one of the Documents of content_type 'table'
print(tables[0].content)
print(tables[0].meta)

  Number of Coils Number of Paperclips
0               5              3, 5, 4
1              10              7, 8, 6
2              15           11, 10, 12
3              20           15, 13, 14
{}


In [24]:
from haystack.nodes.retriever import EmbeddingRetriever

retriever = EmbeddingRetriever(document_store=document_store, embedding_model="deepset/all-mpnet-base-v2-table")

INFO - haystack.modeling.utils -  Using devices: CPU
INFO - haystack.modeling.utils -  Number of GPUs: 0
INFO - haystack.nodes.retriever.dense -  Init retriever using embeddings of model deepset/all-mpnet-base-v2-table
WARNING - haystack.nodes.retriever._embedding_encoder -  You are using a Sentence Transformer with the dot_product function. We recommend using cosine instead. This can be set when initializing the DocumentStore


In [25]:
document_store.update_embeddings(retriever=retriever)

INFO - haystack.document_stores.elasticsearch -  Updating embeddings for all 3 docs ...


Updating embeddings:   0%|          | 0/3 [00:00<?, ? Docs/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

In [26]:
from haystack.nodes import TableReader

reader = TableReader(model_name_or_path="google/tapas-base-finetuned-wtq", max_seq_len=512)

INFO - haystack.modeling.utils -  Using devices: CPU
INFO - haystack.modeling.utils -  Number of GPUs: 0


In [27]:
table_doc = document_store.get_document_by_id("20")
print(table_doc.content)

   Speed (mph)           Driver                         Car     Engine  \
0      407.447  Craig Breedlove           Spirit of America     GE J47   
1      413.199        Tom Green            Wingfoot Express     WE J46   
2       434.22       Art Arfons               Green Monster     GE J79   
3      468.719  Craig Breedlove           Spirit of America     GE J79   
4      526.277  Craig Breedlove           Spirit of America     GE J79   
5      536.712       Art Arfons               Green Monster     GE J79   
6      555.127  Craig Breedlove  Spirit of America, Sonic 1     GE J79   
7      576.553       Art Arfons               Green Monster     GE J79   
8      600.601  Craig Breedlove  Spirit of America, Sonic 1     GE J79   
9      622.407    Gary Gabelich                  Blue Flame     Rocket   
10     633.468    Richard Noble                    Thrust 2  RR RG 146   
11     763.035       Andy Green                  Thrust SSC    RR Spey   

        Date  
0     8/5/63  
1    10

In [28]:
from haystack.utils import print_answers

prediction = reader.predict(query="Who drived Spirit of Green Monster?", documents=[table_doc])
print_answers(prediction, details="all")


Query: Who drived Spirit of Green Monster?
Answers:
[   <Answer {'answer': 'Art Arfons', 'type': 'extractive', 'score': 1.0, 'context':    Speed (mph)           Driver                         Car     Engine  \
0      407.447  Craig Breedlove           Spirit of America     GE J47   
1      413.199        Tom Green            Wingfoot Express     WE J46   
2       434.22       Art Arfons               Green Monster     GE J79   
3      468.719  Craig Breedlove           Spirit of America     GE J79   
4      526.277  Craig Breedlove           Spirit of America     GE J79   
5      536.712       Art Arfons               Green Monster     GE J79   
6      555.127  Craig Breedlove  Spirit of America, Sonic 1     GE J79   
7      576.553       Art Arfons               Green Monster     GE J79   
8      600.601  Craig Breedlove  Spirit of America, Sonic 1     GE J79   
9      622.407    Gary Gabelich                  Blue Flame     Rocket   
10     633.468    Richard Noble                  

In [29]:
print(f"Predicted answer: {prediction['answers'][0].answer}")
print(f"Meta field: {prediction['answers'][0].meta}")

Predicted answer: Art Arfons
Meta field: {'aggregation_operator': 'NONE', 'answer_cells': ['Art Arfons']}


In [30]:
# Initialize pipeline
from haystack import Pipeline

table_qa_pipeline = Pipeline()
table_qa_pipeline.add_node(component=retriever, name="EmbeddingRetriever", inputs=["Query"])
table_qa_pipeline.add_node(component=reader, name="TableReader", inputs=["EmbeddingRetriever"])

In [31]:
prediction = table_qa_pipeline.run("Who drived Spirit of Green Monster?", params={"top_k": 30})
print_answers(prediction, details="minimum")

Batches:   0%|          | 0/1 [00:00<?, ?it/s]


Query: Who drived Spirit of Green Monster?
Answers:
[   {   'answer': 'Art Arfons',
        'context':    Speed (mph)           Driver                         Car     Engine  \
0      407.447  Craig Breedlove           Spirit of America     GE J47   
1      413.199        Tom Green            Wingfoot Express     WE J46   
2       434.22       Art Arfons               Green Monster     GE J79   
3      468.719  Craig Breedlove           Spirit of America     GE J79   
4      526.277  Craig Breedlove           Spirit of America     GE J79   
5      536.712       Art Arfons               Green Monster     GE J79   
6      555.127  Craig Breedlove  Spirit of America, Sonic 1     GE J79   
7      576.553       Art Arfons               Green Monster     GE J79   
8      600.601  Craig Breedlove  Spirit of America, Sonic 1     GE J79   
9      622.407    Gary Gabelich                  Blue Flame     Rocket   
10     633.468    Richard Noble                    Thrust 2  RR RG 146   
11     7